In [ ]:
!pip install accelerate==0.21.0 \
  bitsandbytes==0.40.2 \
  peft==0.5.0 \
  transformers==4.34.0 \
  sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: accelerate
    Found existing installation: acceler

In [ ]:
import pandas as pd
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read Data

In [ ]:

import pandas as pd

In [ ]:
PATH = '/content/drive/MyDrive/ITMO/T5QA'

data = pd.read_csv(PATH + '/data/MedQuAD_clean.csv', index_col=0)
data.index = [i for i in range(len(data))]

data

,question,context,answer,answer_start
0,что такое кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами представляет...,9
1,сколько людей с шерстяными волосами страдают к...,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами встречается ...,9
2,"каковы генетические изменения, связанные с кер...",кератодермия с пушистыми волосами представляе...,"мутации в генах jup, dsp, dsc2 и kank2 вызываю...",2551
3,передается ли кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,большинство случаев кератодермии с курчавыми в...,4573
4,что такое синдром кноблоха?,"синдром кноблоха редкое заболевание, характер...","синдром кноблоха редкое заболевание, характери...",9
...,...,...,...,...
1108,есть ли у вас информация об измерении темпера...,измерение температуры тела может помочь обнару...,измерение температуры тела может помочь обнар...,-1
1109,что такое дефицит аденозинмонофосфатдезаминаз...,дефицит аденозинмонофосфатдезаминазы амф это ...,дефицит аденозинмонофосфат амф дезаминазы это ...,12
1110,сколько людей страдают от дефицита аденозинмо...,дефицит аденозинмонофосфатдезаминазы амф это ...,дефицит амфдезаминазы является одним из наибол...,174
1111,"каковы генетические изменения, связанные с де...",дефицит аденозинмонофосфатдезаминазы амф это ...,мутации в гене ampd1 вызывают дефицит ampдезам...,1495


## Load the Model

In [ ]:
MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

In [ ]:
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()


In [ ]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


In [ ]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=

In [ ]:
data.iloc[ind].question

In [ ]:
data.iloc[3].question

'передается ли кератодермия с пушистыми волосами?'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

inputs = ['сократи текст: "кератодермия с пушистыми волосами представляет собой группу связанных состояний, которые поражают кожу и волосы и во многих случаях повышают риск потенциально опасных для жизни проблем с сердцем. у людей с такими заболеваниями волосы необычно грубые, сухие, тонкие и туго завитые. в некоторых случаях волосы также редкие. шерстистая текстура волос обычно поражает только волосы на коже головы и присутствует с рождения. начиная с раннего возраста, у пораженных людей также развивается ладонноподошвенная кератодермия состояние, при котором кожа на ладонях рук и подошвах ног становится толстой, чешуйчатой ​​и мозолистой. кардиомиопатия, заболевание сердечной мышцы, представляет собой опасную для жизни проблему со здоровьем, которая может развиться у людей с кератодермией и пушистыми волосами. в отличие от других особенностей этого состояния, признаки и симптомы кардиомиопатии могут не проявляться до подросткового возраста или позже. осложнения кардиомиопатии могут включать нарушение сердечного ритма аритмию, сердечную недостаточность и внезапную смерть. кератодермия с пушистыми волосами включает в себя несколько связанных состояний с перекрывающимися признаками и симптомами. исследователи недавно предложили классифицировать кератодермию с пушистыми волосами на четыре типа в зависимости от основной генетической причины. тип i, также известный как болезнь наксоса, характеризуется ладонноподошвенной кератодермией, пушистыми волосами и формой кардиомиопатии, называемой аритмогенной кардиомиопатией правого желудочка адпж. тип ii, также известный как синдром карвахаля, имеет аномалии волос и кожи, аналогичные типу i, но характеризуется другой формой кардиомиопатии, называемой дилатационной кардиомиопатией левого желудочка. тип iii также имеет признаки и симптомы, сходные с симптомами типа i, включая орвк, хотя аномалии волос и кожи часто менее выражены. тип iv характеризуется ладонноподошвенной кератодермией, пушистыми и редкими волосами, а также аномальными ногтями на руках и ногах. тип iv, повидимому, не вызывает кардиомиопатию."']
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 3584,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.5,
  "top_k": 30,
  "top_p": 0.9
}

сократи текст: "кератодермия с пушистыми волосами представляет собой группу связанных состояний, которые поражают кожу и волосы и во многих случаях повышают риск потенциально опасных для жизни проблем с сердцем. у людей с такими заболеваниями волосы необычно грубые, сухие, тонкие и туго завитые. в некоторых случаях волосы также редкие. шерстистая текстура волос обычно поражает только волосы на коже головы и присутствует с рождения. начиная с раннего возраста, у пораженных людей также развивается ладонноподошвенная кератодермия состояние, при котором кожа на ладонях рук и подошвах ног становится толстой, чешуйчатой ​​и мозолистой. кардиомиопатия, заболевание сердечной мышцы, представляет собой опасную для жизни проблему со здоровьем, которая может 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

#inputs = ["Почему трава зеленая?", "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч"]
#inputs = ['сократи текст: "синдром беквитавидемана это нарушение роста, которое приводит к увеличению размеров тела, увеличению органов и другим симптомам. это врожденное заболевание, то есть оно присутствует при рождении. признаки и симптомы расстройства несколько варьируются от ребенка к ребенку. младенчество может быть критическим периодом для детей с этим заболеванием изза возможности: низкого уровня сахара в крови типа грыжи, называемого омфалоцеле при его наличии увеличенного языка макроглоссии повышенной скорости роста опухоли. опухоли вильмса и гепатобластомы являются наиболее распространенными опухолями у детей с этим синдромом."']
special_token = "сократи текст:"

for ind in tqdm(range(len(data))):
    conversation = Conversation()
    conversation.add_user_message(f"{special_token} {data.iloc[ind].answer}")
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)

    with open(PATH + '/data/MedQuAD_answers_sum.csv', "a") as myfile:
        myfile.write(f"{data.iloc[ind].question},{data.iloc[ind].context},{output}")